In [1]:
import os
import sys
from pathlib import Path

# Ensure project root is on sys.path so 'backtester' imports work
cwd = Path(os.getcwd())
candidates = [cwd, cwd.parent, Path("..").resolve()]
for p in candidates:
    if (p / "backtester").exists() and str(p) not in sys.path:
        sys.path.insert(0, str(p))
        break

print("PYTHONPATH set. Using root:", sys.path[0])

PYTHONPATH set. Using root: c:\Users\User\Desktop\Crypto strategy backtest


## Load Data

In [2]:
import pandas as pd

def load_crypto_parquet_data(coin_name: str, timeframe: str = "5m", nM: int = 54, section: str = "UTC") -> pd.DataFrame:
    df = pd.read_parquet(fr'C:\Users\User\Desktop\Crypto\{coin_name}_{timeframe}_{nM}M_{section}.parquet')
    return df
def generate_us_session_bars_info(df, include_holidays: bool = False):
    # 確保時間有時區資訊
    df['dt_ny'] = pd.to_datetime(df['dt_utc'], utc=True).dt.tz_convert('America/New_York')

    # 取日期（當地日曆）
    df['date'] = df['dt_ny'].dt.date
    df['weekday'] = df['dt_ny'].dt.day_name() 
    # 對每天依時間排序並編號
    df = df.sort_values(['date', 'dt_ny']).reset_index(drop=True)
    df['bar_index'] = df.groupby('date').cumcount() + 1  # 第幾根K線，從1開始
    if not include_holidays:
        weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday']
    else:
        weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    df = df.loc[df['weekday'].isin(weekday)]
    # 查看結果
    df.set_index('dt_utc', inplace=True)
    # print(df[['dt_ny', 'date', 'weekday', 'bar_index']].head(5))

    return df
def generate_allday_bars_info(df, include_holidays: bool = True):
# 確保時間有時區資訊
    df['dt_ny'] = pd.to_datetime(df['dt_utc'], utc=True).dt.tz_convert('America/New_York')

    # 取日期（當地日曆）
    df['date'] = df['dt_ny'].dt.date
    df['time'] = df['dt_ny'].dt.time
    df['weekday'] = df['dt_ny'].dt.day_name() 
    # 對每天依時間排序並編號
    df = df.sort_values(['date', 'dt_ny']).reset_index(drop=True)
    df['bar_index'] = df.groupby('date').cumcount() + 1  # 第幾根K線，從1開始
    if not include_holidays:
        weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday']
    else:
        weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    df = df.loc[df['weekday'].isin(weekday)]
    # 查看結果
    df.set_index('dt_utc', inplace=True)
    # print(df[['dt_ny', 'date', 'weekday', 'bar_index']].head(5))

    return df

In [3]:
coin_name = 'BTC'  # 可更改為 'ETH', 'SOL', 'BTC', 'ADA', 'PAXG' 等等
nM = 54
timeframe = "5m"
section = "UTC"

df = load_crypto_parquet_data(coin_name=coin_name, timeframe=timeframe, nM=nM, section=section)
df = generate_allday_bars_info(df, include_holidays=False)


## Split Data

In [4]:
# 分成n等分windows
def split_into_windows(df: pd.DataFrame, n_windows: int, overlap_ratio: float = 0.0) -> list[pd.DataFrame]:
    unique_dates = df.index.unique()
    n_dates = len(unique_dates)
    window_size = n_dates // n_windows
    windows = []
    end_idx = 0
    total_windows = (n_dates - window_size)//(int(window_size * (1 - overlap_ratio))) + 1 if overlap_ratio > 0 else n_windows
    for i in range(total_windows):
        start_idx = end_idx
        if i == total_windows - 1:  # 最後一個視窗包含剩餘的日期
            end_idx = n_dates
        if overlap_ratio > 0 and i > 0:
            overlap_size = int(window_size * overlap_ratio)
            start_idx -= overlap_size
        end_idx = start_idx + window_size
        window_dates = unique_dates[start_idx:end_idx]
        window_df = df[df.index.isin(window_dates)].copy()
        windows.append(window_df)
    return windows
def split_into_train_test(df: pd.DataFrame, train_pct: float = 0.7) -> tuple[pd.DataFrame, pd.DataFrame]:
    unique_dates = df.index.unique()
    n_dates = len(unique_dates)
    train_size = int(n_dates * train_pct)
    train_dates = unique_dates[:train_size]
    test_dates = unique_dates[train_size:]
    train_df = df[df.index.isin(train_dates)].copy()
    test_df = df[df.index.isin(test_dates)].copy()
    return train_df, test_df

In [18]:
splited_dfs = split_into_windows(df, n_windows=27, overlap_ratio=0)
splited_train_test_dfs = [split_into_train_test(window_df, train_pct=0.7) for window_df in splited_dfs]

## Build Strategy

In [14]:
from backtester.models import Position, Side, ActionType, BacktestConfig
from backtester.engine import BacktestEngine
from backtester.strategy_base import StrategyContext
from backtester.strategies.ALBO_strategy import ALBOStrategy, ALBOParams
from typing import Any, Dict, List
from itertools import product
def build_param_combinations(grid: Dict[str, List[Any]]) -> List[Dict[str, Any]]:
    """
    將 param_grid 展開成所有組合，每個組合是一個 dict。
    grid 的值必須是 list/tuple 等 iterable。
    """
    keys = list(grid.keys())
    values_lists = [grid[k] for k in keys]
    combos = [dict(zip(keys, vals)) for vals in product(*values_lists)]
    return combos
def tune_strategy_params(train_df: pd.DataFrame, param_grid: dict, best_n_params: int, engine) -> dict:
    best_params = None
    best_performance = -float('inf')
    params_combinations = build_param_combinations(param_grid)
        
    performance_list = []

    for params in params_combinations:
        strat_params = ALBOParams(**params)
        strat = ALBOStrategy(strat_params)
        train_result = engine.run(train_df, strat)
        performance = train_result.equity_curve.values[-1] - train_result.equity_curve.values[0]  # 總收益作為績效指標
        unique_days = train_df.index.normalize().unique()
        performance_per_day = performance / (len(unique_days))

        if performance_per_day > best_performance:
            best_performance = performance_per_day
        performance_list.append((params, performance_per_day, len(train_result.trades)))
    performance_list.sort(key=lambda x: x[1], reverse=True)
    best_params_list = performance_list[:best_n_params]

    return best_params_list
def test_strategy_with_params(test_df: pd.DataFrame, params: dict, engine) -> Any:
    strat_params = ALBOParams(**params)
    strat = ALBOStrategy(strat_params)
    test_result = engine.run(test_df, strat)
    performance = test_result.equity_curve.values[-1] - test_result.equity_curve.values[0]
    unique_days = test_df.index.normalize().unique()
    performance_per_day = performance / (len(unique_days))
    return performance_per_day, test_result


In [7]:
cfg = BacktestConfig(initial_cash=10000, fee_rate=0.0, slippage_bps=0.0, conservative_intrabar=True)
engine = BacktestEngine(cfg)

In [8]:
for param, values in ALBOParams.__annotations__.items():
      print(f"Parameter: {param} | Type: {values}")  # 查看可調參數

Parameter: break_out_series_n | Type: int
Parameter: break_out_n_bars | Type: int
Parameter: BO_n_times_atr | Type: float
Parameter: max_notional_pct | Type: float
Parameter: min_qty | Type: float
Parameter: sl_atr_like | Type: float
Parameter: fixed_sl_pct | Type: float
Parameter: rr | Type: float
Parameter: time_exit_bars | Type: int


In [20]:

param_grid = {
    'rr': [1.5],
    'break_out_n_bars': [10, 20, 30],
    'break_out_series_n': [2, 3, 4, 5],
    'BO_n_times_atr': [0.5, 1.0, 1.5],
}
# param_grid = {
#     'rr': [1.5],
#     'break_out_n_bars': [10],
#     'break_out_series_n': [2],
#     'BO_n_times_atr': [0.5],
# }
best_n_params = 2
best_params_list = []
start_window = 10
for i in range(10, len(splited_train_test_dfs)):
    print(f"=== Window {i+1} ===")
    best_params = tune_strategy_params(
        train_df=splited_train_test_dfs[i][0],
        param_grid=param_grid,
        best_n_params=best_n_params,
        engine=engine
    )
    print("Best Params:")
    for params, perf, trades in best_params:
        print(f"Params: {params}, Performance per day: {perf}, Trades: {trades}")
    best_params_list.append(best_params)



=== Window 11 ===
Best Params:
Params: {'rr': 1.5, 'break_out_n_bars': 10, 'break_out_series_n': 3, 'BO_n_times_atr': 1.0}, Performance per day: 25.30679218052845, Trades: 28
Params: {'rr': 1.5, 'break_out_n_bars': 10, 'break_out_series_n': 3, 'BO_n_times_atr': 0.5}, Performance per day: 24.70627754398511, Trades: 53
=== Window 12 ===
Best Params:
Params: {'rr': 1.5, 'break_out_n_bars': 10, 'break_out_series_n': 2, 'BO_n_times_atr': 1.0}, Performance per day: 20.264694695547774, Trades: 122
Params: {'rr': 1.5, 'break_out_n_bars': 20, 'break_out_series_n': 2, 'BO_n_times_atr': 1.0}, Performance per day: 18.621065563667898, Trades: 103
=== Window 13 ===
Best Params:
Params: {'rr': 1.5, 'break_out_n_bars': 30, 'break_out_series_n': 3, 'BO_n_times_atr': 1.5}, Performance per day: 3.0205418824687613, Trades: 13
Params: {'rr': 1.5, 'break_out_n_bars': 20, 'break_out_series_n': 3, 'BO_n_times_atr': 1.5}, Performance per day: 2.6211548268327878, Trades: 14
=== Window 14 ===
Best Params:
Params

In [ ]:

param_grid = {
    'rr': [1.5],
    'break_out_n_bars': [10, 20, 30],
    'break_out_series_n': [2, 3, 4, 5],
    'BO_n_times_atr': [0.5, 1.0, 1.5],
}
best_n_params = 2
best_params_list = []
for i in range(start_window, len(splited_train_test_dfs)):
    print(f"=== Window {i+1} ===")
    best_params = tune_strategy_params(
        train_df=splited_train_test_dfs[i][1],
        param_grid=param_grid,
        best_n_params=best_n_params,
        engine=engine
    )
    print("Best Params:")
    for params, perf, trades in best_params:
        print(f"Params: {params}, Performance per day: {perf}, Trades: {trades}")
    best_params_list.append(best_params)



In [23]:
best_params = [
    ({'rr': 1.5, 'break_out_series_n': 5},{'rr': 1.2, 'break_out_series_n': 5}),
    {'rr': 1, 'break_out_series_n': 5},
    {'rr': 1.5, 'break_out_series_n': 2},
    {'rr': 1.5, 'break_out_series_n': 2},
    {'rr': 1, 'break_out_series_n': 2}
]

for i in range(start_window, len(splited_train_test_dfs)):
    print(f"=== Window {i+1} Test Results ===")
    for params, train_perf, _ in best_params_list[i-start_window]:
        test_perf, test_result = test_strategy_with_params(
            test_df=splited_train_test_dfs[i][1],
            params=params,
            engine=engine
        ) 
        print(f"Test Result with Params: {params}, performance per day: {test_perf}, Trades: {len(test_result.trades)} , performance diff: {test_perf - train_perf}")

=== Window 11 Test Results ===
Test Result with Params: {'rr': 1.5, 'break_out_n_bars': 10, 'break_out_series_n': 3, 'BO_n_times_atr': 1.0}, performance per day: -16.908221787801427, Trades: 20 , performance diff: -42.21501396832988
Test Result with Params: {'rr': 1.5, 'break_out_n_bars': 10, 'break_out_series_n': 3, 'BO_n_times_atr': 0.5}, performance per day: -14.933700587525356, Trades: 27 , performance diff: -39.63997813151047
=== Window 12 Test Results ===
Test Result with Params: {'rr': 1.5, 'break_out_n_bars': 10, 'break_out_series_n': 2, 'BO_n_times_atr': 1.0}, performance per day: 6.712271054222356, Trades: 46 , performance diff: -13.552423641325419
Test Result with Params: {'rr': 1.5, 'break_out_n_bars': 20, 'break_out_series_n': 2, 'BO_n_times_atr': 1.0}, performance per day: 4.700766369083681, Trades: 39 , performance diff: -13.920299194584217
=== Window 13 Test Results ===
Test Result with Params: {'rr': 1.5, 'break_out_n_bars': 30, 'break_out_series_n': 3, 'BO_n_times_atr

## analytics

In [28]:
cfg = BacktestConfig(initial_cash=10000, fee_rate=0.0, slippage_bps=0.0, conservative_intrabar=True)
engine = BacktestEngine(cfg)

In [ ]:
from backtester.analytics import basic_metrics
from dataclasses import replace
params = ALBOParams(
    break_out_series_n = 3, # BO定義:連續突破K線數
    break_out_n_bars = 20, # 突破區間長度（K線數）
    BO_n_times_atr= 1.5,
    max_notional_pct = 1.0, # 最大虧損佔資金比例%
    min_qty = 0.001, # 最小BTC下單量
    sl_atr_like = 0.0,  # MVP不做ATR，示範保留欄位
    rr = 1.5,           # TP = SL距離 * rr
    time_exit_bars = 50
)
for fee_rate in [0.0, 0.00005, 0.0001]:
    strategy = ALBOStrategy(params)
    cfg2 = replace(cfg, fee_rate=fee_rate)
    engine = BacktestEngine(cfg2)
    print(engine.config.fee_rate)
    result = engine.run(df, strategy)
    metrics = basic_metrics(result)
    print(f"=== Fee Rate: {fee_rate} ===")
    for k, v in metrics.items():
        print(f"{k}: {v}")

0.0
=== Fee Rate: 0.0 ===
trades: 687.0
win_rate: 0.42066957787481807
avg_pnl: 8.263892501638983
profit_factor: 1.1757962459638076
max_drawdown: -0.2787439335822491
0.5
=== Fee Rate: 0.5 ===
trades: 687.0
win_rate: 0.0
avg_pnl: -4995.868053749181
profit_factor: 0.0
max_drawdown: -686.7161352925691


In [27]:
metrics = basic_metrics(result)
for k, v in metrics.items():
    print(f"{k}: {v}")

trades: 687.0
win_rate: 0.42066957787481807
avg_pnl: 8.263892501638983
profit_factor: 1.1757962459638076
max_drawdown: -0.2787439335822491


## 